

> ## AI-Powered Stock Investment Optimization :

## Overview:
Stock Market Investment Optimization is an AI-powered application that helps users allocate their investments across multiple companies to maximize return on investment (ROI). The system considers real-time stock market data, individual risk tolerance, and investment goals to provide optimized investment recommendations.

## Features
- **Real-time Stock Data Collection**: Uses Yahoo Finance's API to fetch stock prices, financial news, and market trends.
- **User Input Handling**: Allows users to specify investment capital, risk level (conservative, moderate, aggressive), investment duration, and industry preferences.

## Application Components
1. **Data Collection & Processing**:
   - Fetch historical and real-time stock market data.
   - Analyze stock performance metrics (P/E ratio, market cap, price fluctuations, etc.).
   - Research market trends and economic forecasts.
2. **Optimization Algorithms**:
   - Implement and compare Greedy Search, A*, and Simulated Annealing.
   - Model portfolio allocation as a CSP.
3. **Investment Strategy Dashboard**:
   - User-friendly interface for inputting investment parameters.
   - Portfolio allocation suggestions with ROI and risk assessment.
   - Visualizations: Portfolio breakdown, risk-return trade-offs, and historical performance graphs.

## Technologies Used
- **Python** (Jupyter Notebook for implementation)
- **APIs** (Alpha Vantage, Yahoo Finance)
- **AI Algorithms** (Greedy, A*, Simulated Annealing, CSP)
- **Visualization** (Matplotlib, Seaborn, Plotly)


---
## **1. Problem Formulation**
---

## Portfolio Optimization with A*/Greedy Search Over Time

We model the problem as a **graph**, where each time step `Tᵢ` (for `i ∈ [0, N]`) represents a moment in time.

At each `Tᵢ`, we perform an **A\* search** or **greedy search**, while maintaining:
- The **heuristic values** from previous steps.
- The **portfolio state** from the previous time.

---

### 🔷 Graph Modeling


- **Nodes** represent an individual from stocks  at a given time.
- **Edges** represent possible trades: **Buy**, **Sell**, or **Hold** between stocks .


## Class Definition :

In [12]:
class StockItem:
    """
    Represents an individual stock item with a stock name and the amount owned.
    """
    def __init__(self, stock, amt=1):
        """
        Initializes a stock item.

        :param stock: The name of the stock.
        :param amt: The amount of the stock owned (default is 1).
        """
        self.stock = stock
        self.amt = amt

class Stocks:
    """
    Represents a stock portfolio where users can buy and sell stocks using available funds.
    """
    def __init__(self, funds, stocks=None):
        """
        Initializes the Stocks portfolio.

        :param funds: Initial amount of funds available.
        :param stocks: Optional list of StockItem objects representing owned stocks.
        """
        self.funds = funds
        self.stocks = stocks if stocks is not None else []
        self.value  = 0

    def get_stocks(self):
        return self.stocks
    def get_funds(self):
      return self.funds
    def buy(self, stock, price , show_res  = True):
        """
        Purchases a stock if sufficient funds are available.

        :param stock: The name of the stock to buy.
        :param price: The price of the stock.
        """
        if self.funds < price:
            print(f"Insufficient funds to buy {stock}. Available: ${self.funds:.2f}, Required: ${price:.2f}")
            return

        self.funds -= price  # Deduct the stock price from funds
        found = False

        # Check if the stock is already owned
        for el in self.stocks:
            if el.stock == stock:
                el.amt += 1  # Increase the stock amount
                found = True
                break

        # If stock is not found, add a new entry
        if not found:
            self.stocks.append(StockItem(stock))
        if show_res:
          print(f"Bought 1 share of {stock} for ${price:.2f}. Remaining funds: ${self.funds:.2f}")

    def sell(self, stock, price , show_res  = True):
        """
        Sells a stock if the user owns it.

        :param stock: The name of the stock to sell.
        :param price: The price of the stock at the time of selling.
        """
        for el in self.stocks:
            if el.stock == stock:
                el.amt -= 1  # Reduce stock amount
                self.funds += price  # Add the selling price to funds

                # Remove the stock from the list if the amount reaches zero
                if el.amt == 0:
                    self.stocks.remove(el)

                if show_res:
                  print(f"Sold 1 share of {stock} for ${price:.2f}. Updated funds: ${self.funds:.2f}")
                return

        print(f"Cannot sell {stock}: Not owned.")

    def has_stock(self, stock):
        """
        Checks if a stock exists in the portfolio.

        :param stock: The name of the stock to check.
        :return: True if the stock is in the portfolio, False otherwise.
        """
        return any(el.stock == stock for el in self.stocks)
    def get_stock (self , stock):
      for st in self.stocks :
        if st.stock == stock :
          return st.amt
      return 0
    def eval_por(self, stocks):
      prix = 0
      for stock in stocks :
        if self.has_stock(stock["symbol"]):
          prix = prix + self.get_stock(stock)
      return prix

    def percentage(self, stock):
        """
        Calculates the percentage of a specific stock in the portfolio.

        :param stock: The name of the stock to calculate percentage for.
        :return: The percentage of the stock in the portfolio, or 0 if not found or total is zero.
        """
        if not self.has_stock(stock):
            return 0.0

        total = sum(el.amt for el in self.stocks)  # Total number of shares
        if total == 0:  # Prevent division by zero
            return 0.0

        for el in self.stocks:
            if el.stock == stock:
                return el.amt / total
        return 0.0  # Fallback, though has_stock ensures this shouldn't happen

    def display_portfolio(self):
        """
        Displays the current stock holdings and available funds.
        """
        print(f"\nAvailable funds: ${self.funds:.2f}")
        if not self.stocks:
            print("Stock Portfolio is empty.")
        else:
            print("Stock Portfolio:")
            for stock in self.stocks:
                print(f"- {stock.stock}: {stock.amt} shares")
        print("-" * 30)
    def transition(self, funds, dayPrices):
    # Create price lookup dictionary
      prices = {price["symbol"]: price['price'] for price in dayPrices}

      # Create stock information dictionary
      stck = {}
      for stock in self.stocks:
          symbol = stock.stock
          stck[symbol] = {
              "price": prices[symbol],
              "amt": self.get_stock(symbol),
              "per": self.percentage(symbol)
          }

      # Update funds
      self.funds = funds

      # Clear current stocks (after we've used them for calculations)
      self.stocks = []

      # Rebuild portfolio according to percentages
      for symbol, det in stck.items():
          if det["per"] <= 0:  # Skip if percentage is 0 or negative
              continue

          # Calculate amount to buy (using available funds)
          amount_to_spend = funds * det["per"]
          num_to_buy = int(amount_to_spend // det["price"])

          # Execute buys
          for _ in range(num_to_buy):
              if self.funds >= det["price"]:  # Check if we can afford it
                  self.buy(symbol, det["price"])
              else:
                  break  # Stop if we run out of money

    def __lt__(self, other):
        # Define how nodes should be compared
        # For example, compare by heuristic value:
        return self.get_heu() < other.get_heu()

    # Optionally implement other comparison methods
    def __eq__(self, other):
        return self.get_heu() == other.get_heu()



## Structring  the obtained data :

# retrieving stock prices
Uses the yfinance API to fetch historical stock prices between two dates.

*   Retrieves 1-minute interval data for periods up to 7 days  
*  Uses 5-minute intervals for date ranges between 7 days and 2 months  
* Fetches daily prices for intervals within a 1-year span  
* Gets weekly prices for time ranges longer than 1 year
     
Returns a list of dictionaries in this format:

    {
        'symbol': "AAPL",      # stock ticker symbol
        'prices': [190.2, 191.5, ...]  # list of price values (floats)
    }
    





In [13]:
!pip install yfinance --upgrade
import yfinance as yf
from datetime import datetime, timedelta
import time
import json

def get_stock_data(stock_symbols, start_date, end_date):
    """
    Fetch stock data for given symbols between two dates.

    Args:
        stock_symbols (list): List of stock symbols (e.g., ['AAPL', 'MSFT']).
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        list: A list of dictionaries containing stock symbols and their close prices.
    """
    # Convert string dates to datetime objects
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')

    # Calculate the time delta between start and end dates
    delta = end - start

    # Determine the appropriate interval based on the time range
    if delta <= timedelta(days=7):
        interval = '1m'  # 1 minute for intraday data (max 7 days)
    elif delta <= timedelta(days=60):
        interval = '5m'  # 5 minutes for short-term
    elif delta <= timedelta(days=365):
        interval = '1d'  # daily for medium-term
    else:
        interval = '1wk'  # weekly for long-term

    results = []

    for stock in stock_symbols:
        # Fetch stock data using yfinance
        stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
        # Extract close prices as a list
        close_dic = stock_data['Close'].to_dict().values()
        close_prices =list(key.values() for key in close_dic)
        prix = []
        for el in close_prices :
          for val in el:
             prix.append(val)


        # Append the result for this stock to the results list
        results.append({
            'symbol': stock ,
            'prices': prix
        })

    return results



---
1.Node representation
---

In [14]:
class Node:
    """
    Represents a node in a graph, typically used in search algorithms like A* or BFS.

    Attributes:
        value (float): The value associated with this node (e.g., stock price).
        heuristic (float): The heuristic value estimating the cost to reach the goal.
        symbol (str): A unique identifier for the node (e.g., stock symbol).
        action (str): The action associated with this node (e.g., 'buy', 'sell', 'hold').
        neighbors (list): A list of neighboring nodes connected to this node.
        parent (Node): The parent node in the search tree.
    """

    def __init__(self, stock, heuristic, parent=None, neighbors=None):
        """
        Initializes a Node object.

        :param stock: A dictionary or object containing stock details (symbol, action, value).
        :param heuristic: The heuristic value for the node.
        :param parent: The parent node in the search tree (default is None).
        :param neighbors: A list of neighboring nodes (default is an empty list).
        """
        self.value = stock.get("value", 0)  # Cost of the stock (default to 0 if not provided)
        self.heuristic = heuristic  # Heuristic value
        self.symbol = stock.get("symbol", "unknown")  # Stock symbol (default to 'unknown' if not provided)
        self.action = stock.get("action", "none")  # Action (default to 'none' if not provided)
        self.neighbors = neighbors if neighbors is not None else []  # Neighbors list
        self.parent = parent  # Parent node

    @property
    def set_cop_stock(self):
        """Returns a dictionary representation of the stock data."""
        return {
            "symbol": self.symbol,
            "action": self.action,
            "value": self.value
        }

    def add_neighbor(self, neighbor):
        """
        Adds a neighboring node to this node's neighbors list.

        :param neighbor: The neighboring node to add.
        """
        if neighbor not in self.neighbors:
            self.neighbors.append(neighbor)

    def remove_neighbor(self, neighbor):
        """
        Removes a neighboring node from this node's neighbors list.

        :param neighbor: The neighboring node to remove.
        """
        if neighbor in self.neighbors:
            self.neighbors.remove(neighbor)

    def set_parent(self, parent):
        """Sets the parent node."""
        self.parent = parent

    def get_Key(self):
        """Returns a unique key for the node combining symbol and action."""
        return f"{self.symbol}_{self.action}"

    def get_heu(self):
        """Returns the heuristic value of the node."""
        return self.heuristic

    def get_copy(self):
        """Returns a copy of the node."""
        copy_node = Node(self.set_cop_stock, self.heuristic)
        return copy_node

    def __repr__(self):
        """
        String representation of the node for debugging purposes.

        :return: A string summarizing the node's attributes.
        """
        return (
            f"Node(symbol={self.symbol}, action={self.action}, value={self.value:.2f}, "
            f"heuristic={self.heuristic:.2f}, neighbors={len(self.neighbors)})"
        )

## Heursitc Function :


The heuristic function combines various financial metrics to guide portfolio optimization. Each metric plays a specific role in selecting stocks that align with the investor's risk and return preferences.

- **Sharpe Ratio**  
  Prioritizes stocks with high **risk-adjusted returns**. It considers both the excess return over a risk-free rate and the total volatility of returns.

- **Sortino Ratio**  
  Focuses on **downside risk** by isolating harmful volatility, making it more appropriate for risk-averse strategies.

- **Value at Risk (VaR)**  
  Estimates the **maximum expected loss** at a given confidence level under normal market conditions.

- **Conditional Value at Risk (CVaR)**  
  Measures the **average loss** beyond the VaR threshold, capturing extreme tail risks to protect against **catastrophic losses**.

- **Momentum**  
  Indicates short-term or long-term **price movement strength**, helping identify bullish trends.

- **MACD (Moving Average Convergence Divergence)**  
  Highlights **trend reversals** and **momentum shifts** using EMAs.

- **RSI (Relative Strength Index)**  
  Identifies **overbought or oversold** conditions to avoid poorly timed entries or exits.




In [15]:
import random
import numpy as np
from scipy.stats import norm

# Define the list of stock symbols (you can customize this list)
stock_symbols = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "NVDA", "META", "NFLX",
    "AMD", "INTC", "PYPL", "ADBE", "CRM", "AVGO", "QCOM", "TXN",
    "CSCO", "IBM", "ORCL", "VRTX"
]

# Generate the stocks list with 20 stocks, each having 50 random prices
def generate_stock_data():
    stocks = []
    for symbol in stock_symbols:
        # Generate 50 random prices between 50 and 500
        prices = [round(random.uniform(50, 500), 2) for _ in range(80)]
        stocks.append({
            "symbol": symbol,
            "prices": prices
        })
    return stocks

def calculate_returns(prices):
    """Calculate daily returns from a list of prices."""
    returns = np.diff(prices) / prices[:-1]
    return returns

def get_var_and_cvar(stock, confidence_level=0.95):
    """
    Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR).

    Args:
        stock (dict): Dictionary containing 'symbol' and 'prices'.
        confidence_level (float): Confidence level for VaR (e.g., 0.95 for 95%).

    Returns:
        tuple: VaR and CVaR values.
    """
    prices = stock["prices"]
    returns = calculate_returns(prices)

    z_score = norm.ppf(1 - confidence_level)  # Z-score for the given confidence level
    mean = np.mean(returns)
    std_dev = np.std(returns)
    var = mean - z_score * std_dev  # VaR formula

    # Calculate CVaR as the mean of losses exceeding VaR
    losses_beyond_var = returns[returns <= var]
    cvar = np.mean(losses_beyond_var) if len(losses_beyond_var) > 0 else var

    return var, cvar  # Return raw VaR/CVaR (negative for losses)

def get_sharpe_ratio(stock, risk_free_rate=0.02, trading_days=252):
    """
    Calculate the Sharpe Ratio for each stock.

    Args:
        stock: dictionary 'symbol' and 'prices'.
        risk_free_rate (float): Annual risk-free rate (default 2%).
        trading_days (int): Number of trading days in the dataset.

    Returns:
        dict: Dictionary with stock symbols as keys and their Sharpe ratio as values.
    """
    symbol = stock["symbol"]
    prices = stock["prices"]
    returns = calculate_returns(prices)

    mean_daily_return = np.mean(returns)
    annual_excess_return = (mean_daily_return * trading_days) - risk_free_rate
    annual_std_dev = np.std(returns) * np.sqrt(trading_days)

    sharpe = annual_excess_return / annual_std_dev if annual_std_dev != 0 else float('inf')
    return round(sharpe, 4) # Clamp negative values to 0

def get_sortino_ratio(stock, risk_free_rate=0.02, trading_days=252):
    """
    Calculate the Sortino Ratio for each stock.

    Args:
        stock: dictionary 'symbol' and 'prices'.
        risk_free_rate (float): Annual risk-free rate (default 2%).
        trading_days (int): Number of trading days in the dataset.

    Returns:
        dict: Dictionary with stock symbols as keys and their Sortino ratio as values.
    """
    symbol = stock["symbol"]
    prices = stock["prices"]
    returns = calculate_returns(prices)

    mean_daily_return = np.mean(returns)
    annual_excess_return = (mean_daily_return * trading_days) - risk_free_rate

    downside_returns = returns[returns < 0]
    downside_deviation = np.std(downside_returns) * np.sqrt(trading_days) if len(downside_returns) > 0 else 0

    sortino = annual_excess_return / downside_deviation if downside_deviation != 0 else float('inf')
    return  round( sortino, 4)  # Clamp negative values to 0

def calculate_ema(prices, span):
    """
    Calculate Exponential Moving Average (EMA).

    Args:
        prices (list): List of stock prices.
        span (int): Span for the EMA (e.g., 12, 26, or 9).

    Returns:
        list: EMA values.
    """
    if not prices or span <= 0:
        raise ValueError("Prices must be non-empty and span must be positive.")

    # Calculate the smoothing factor (alpha)
    alpha = 2 / (span + 1)

    # Initialize the EMA array with the first price
    ema = [prices[0]]

    # Calculate EMA for each subsequent day
    for i in range(1, len(prices)):
        ema_today = prices[i] * alpha + ema[-1] * (1 - alpha)
        ema.append(ema_today)

    return ema
def get_RSI(stock, period=14):
    """
    Calculate the Relative Strength Index (RSI) for a given stock.

    Parameters:
        stock (dict): A dictionary containing stock data, including "prices".
        period (int): The number of periods to use for RSI calculation (default is 14).

    Returns:
        float: The RSI value (between 0 and 100).
    """
    prices = stock["prices"]

    # Ensure there are enough prices to calculate RSI
    if len(prices) <= period:
        raise ValueError("Not enough price data to calculate RSI. Need at least `period + 1` prices.")

    # Step 1: Calculate price changes
    price_changes = [prices[i] - prices[i - 1] for i in range(1, len(prices))]

    # Step 2: Separate gains and losses
    gains = [max(change, 0) for change in price_changes]  # Positive changes or 0
    losses = [-min(change, 0) for change in price_changes]  # Absolute values of negative changes or 0

    # Step 3: Calculate average gains and losses over the specified period
    avg_gain = np.mean(gains[:period])  # SMA of first N gains
    avg_loss = np.mean(losses[:period])  # SMA of first N losses

    # Avoid division by zero
    if avg_loss == 0:
        return 100  # RSI is 100 when there are no losses

    # Step 4: Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Step 5: Calculate RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi
def get_MacD(stock):
    prices = stock["prices"]
    if len(prices) < 26:
        return None
    shortEma = calculate_ema(prices[-12:], 12)
    longEma = calculate_ema(prices[-26:], 26)
    macd = [longEma[i] - shortEma[i] for i in range(12)]
    signal_line = calculate_ema(macd[-9:], 9)
    hist = [macd[-9:][i] - signal_line[i] for i in range(9)]
    return round(hist[-1] , 4)
def calc_Momentum(stock , short = False):
    prices = stock['prices']
    T = len(prices)
    n = T//6 if short else T//4
    ti = prices[-1]
    tn = prices[-n]
    return  round((ti - tn )/ tn , 4)

In [16]:
def calculate_heuristics(stock, market_data=None):
    """
    Returns balanced buy/hold/sell scores with all metrics fairly weighted.
    Key improvements:
    1. Normalizes all inputs to [0, 1] using empirical bounds or sigmoid/tanh transformations.
    2. Rebalances weights to prevent any single metric (e.g., Sharpe, VaR) from dominating.
    3. Adds dynamic scaling for momentum, MACD, and RSI.
    4. Incorporates all available metrics: Momentum, MACD, Sharpe, Sortino, VaR/CVaR, RSI, Stability.
    """
    symbol = stock["symbol"]

    # --- Step 1: Normalize All Inputs ---
    # Momentum: Directional strength (tanh bounds to [-1, 1], then shift to [0, 1])
    momentum_short = (np.tanh(calc_Momentum(stock, True)) + 1) / 2  # [0, 1]
    momentum_long = (np.tanh(calc_Momentum(stock, False)) + 1) / 2   # [0, 1]

    # MACD: Sigmoid to bound to [0, 1] (positive = bullish, negative = bearish)
    macd = get_MacD(stock)
    if macd is not None:
        macd_score = 1 / (1 + np.exp(-macd))  # [0, 1]
        neg_macd_score = 1 / (1 + np.exp(macd))  # Flipped for sell signals
    else:
        macd_score = 0.5  # Neutral if MACD cannot be calculated
        neg_macd_score = 0.5

    # RSI: Normalize to [0, 1] (RSI > 70 = overbought, RSI < 30 = oversold)
    rsi = get_RSI(stock)
    rsi_normalized = max(0, min((rsi - 30) / 40, 1))  # Scale RSI to [0, 1]

    # Sharpe/Sortino: Percentile rank (if market_data exists) or softmax scaling
    sharpe = max(0, get_sharpe_ratio(stock))
    sortino = max(0, get_sortino_ratio(stock))
    if market_data:
        sharpe_rank = np.sum(np.array(market_data["sharpes"]) <= sharpe) / len(market_data["sharpes"])
        sortino_rank = np.sum(np.array(market_data["sortinos"]) <= sortino) / len(market_data["sortinos"])
    else:
        sharpe_rank = 1 - np.exp(-0.3 * sharpe)  # Diminishing returns for high Sharpe
        sortino_rank = 1 - np.exp(-0.3 * sortino)

    # VaR/CVaR: Convert risk to "safety" (1 = safest, 0 = extreme risk)
    var, cvar = get_var_and_cvar(stock)
    var_safety = max(0, min(1 + var / 0.3, 1))  # Assume max VaR = -30%
    cvar_safety = max(0, min(1 + cvar / 0.5, 1))  # Assume max CVaR = -50%

    # Stability: Already in [0, 1] (no change)
    stability_short = 1 - min(abs(momentum_short - 0.5) * 2, 1)  # Centered around 0.5
    stability_long = 1 - min(abs(momentum_long - 0.5) * 2, 1)

    # --- Step 2: Rebalanced Weights ---
    # Short-Term Scores
    buy_score_short = (
        0.3 * momentum_short +  # Primary driver
        0.2 * macd_score +      # Trend confirmation
        0.2 * rsi_normalized +  # Overbought/oversold indicator
        0.2 * sharpe_rank +    # Risk-adjusted return filter
        0.05 * var_safety +      # Risk safety
        0.05 * stability_short  # Stability weight
    )

    sell_score_short = (
        0.3 * (1 - momentum_short) +  # Weak momentum
        0.2 * neg_macd_score +        # Bearish trend
        0.2 * (1 - rsi_normalized) +  # Overbought signal
        0.15 * (1 - var_safety) +     # High risk
        0.1 * (1 - cvar_safety) +     # Extreme risk
        0.05 * (1 - stability_short)  # Low stability
    )

    hold_score_short = (
        0.3 * sharpe_rank +
        0.3 * sortino_rank +
        0.2 * stability_short +  # Higher weight on stability
        0.1 * rsi_normalized +    # Neutral RSI preference
        0.1 * var_safety          # Mild risk preference
    )

    # Long-Term Scores
    buy_score_long = (
        0.3 * momentum_long +
        0.2 * macd_score +
        0.2 * rsi_normalized +
        0.15 * sharpe_rank +
        0.1 * var_safety +
        0.05 * stability_long
    )

    sell_score_long = (
        0.3 * (1 - momentum_long) +
        0.2 * neg_macd_score +
        0.2 * (1 - rsi_normalized) +
        0.15 * (1 - var_safety) +
        0.1 * (1 - cvar_safety) +
        0.05 * (1 - stability_long)
    )

    hold_score_long = (
        0.3 * sharpe_rank +
        0.3 * sortino_rank +
        0.2 * stability_long +  # Higher stability weight for long-term
        0.1 * rsi_normalized +
        0.1 * var_safety
    )

    # --- Step 3: Dynamic Scaling ---
    # Ensure no score exceeds 1.0 due to rounding errors
    def clamp(x): return max(0, min(1, x))

    return {
        "symbol": symbol,
        "buy_short": clamp(buy_score_short),
        "sell_short": clamp(sell_score_short),
        "hold_short": clamp(hold_score_short),
        "buy_long": clamp(buy_score_long),
        "sell_long": clamp(sell_score_long),
        "hold_long": clamp(hold_score_long),
    }

## Cost Function :

In [17]:
def evaluate(current_node , portfolio , prices):
    """
    Evaluate the total portfolio value of a given state.
    Parameters:
    - state (dict): Dictionary with stock symbols as keys and info as values.
    Returns:
    - float: Total evaluated portfolio value.
    """
    path=get_path(current_node)
    total_value = 0
    available_funds= portfolio.funds
    prices = {price["symbol"]:price["price"] for price in prices}
    for stock_data in portfolio.stocks:
        shares = stock_data.amt
        price = prices [stock_data.stock]
        total_value += shares * price

    for node in path:
        if node.action == "sell":
            total_value-= node.value
        elif node.action=="buy":
            total_value+= node.value

    for node in path:
        if node.action == "sell":
            available_funds += node.value
        elif node.action =="buy":
            available_funds -= node.value

    return total_value ,available_funds

def get_path(current_node):
    path=[]
    while not current_node is None :
        path.append(current_node)
        current_node=current_node.parent
    return path[::-1]


---
## **2. Search Algorithm**



 - **Construction of the state  space**


In [18]:
import copy

def setNodes(expansion, history, short=False , sellatonce = False , portfolio = None):
    """
    Creates a graph of nodes based on the expansion list.

    :param expansion: A list of dictionaries representing possible actions for each stock.
    :param history: Historical data used for heuristic calculation.
    :param short: Boolean indicating if short positions are considered.
    :return: A dictionary mapping unique node identifiers to their respective Node objects.
    """
    # Create base node
    baseNode = Node(
        {"symbol": "emp", "action": "none", "value": 0},
        0,
        None,
        neighbors=[]
    )

    nodes = [ baseNode ]
    prev_nodes = [baseNode]

    for stock_info in expansion:
        stock_id = stock_info["symbol"]
        stock_heuristic = calculate_heuristics(history[stock_id])
        created_nodes = []
        num = 0
        if sellatonce and portfolio:
            for stock in portfolio.stocks :
              if stock.stock == stock_id:
                num = stock.amt - 1
        # Helper function to create nodes
        def create_node(action, value, heuristic_key):
            if action in stock_info:
                node = Node(
                    {"symbol": stock_id, "action": action, "value": value},
                    stock_heuristic[heuristic_key],
                    None,
                    neighbors=[]
                )

                created_nodes.append(node)
                nodes.append( node )
        # Create nodes for each action
        create_node("buy", -stock_info.get("buy", 0),
                  "buy_short" if short else "buy_long")
        create_node("sell", stock_info.get("sell", 0),
                   "sell_short" if short else "sell_long")
        create_node("hold", stock_info.get("hold", 0),
                   "hold_short" if short else "hold_long")

        # Connect nodes
        new_prev_nodes = []
        for prev_node in prev_nodes:
            for new_node in created_nodes:
                node_copy = new_node.get_copy()
                prev_node.neighbors.append(node_copy)
                node_copy.set_parent(prev_node)
                new_prev_nodes.append(node_copy)

        prev_nodes = new_prev_nodes
        for _ in range(num):
          create_node("sell", stock_info.get("sell", 0),
                   "sell_short" if short else "sell_long")
          create_node("hold", stock_info.get("hold", 0),
                   "hold_short" if short else "hold_long")
          new_prev_nodes = []
          for prev_node in prev_nodes:
            for new_node in created_nodes:
                node_copy = new_node.get_copy()
                prev_node.neighbors.append(node_copy)
                node_copy.set_parent(prev_node)
                new_prev_nodes.append(node_copy)
          prev_nodes = new_prev_nodes
    return nodes



- **Expension Function**


### Overview
- This part focuses on implementing the expantion function and all the needed helper funciton. These will serve as a core component in the search algorithms that will be implemented in the search notebook including A*, greedy , Simulated Annealing.

- The expansion funtion contains:

   **get_valid_actions (state, mainstock, diversification)** : this function is responsible of returning a list of the valid actions (buy,sell,hold) based on constraints.
   
   **generate_random_neighbor (neighbors)**: this function is resposible of choosing a random neighbor from a given list of neighbors (will be used in simulated annealing).
   
   **best_neighbor (neighbors)** : this function is resposible of choosing the best neighbor and returning it as a Candidate object.
   
   **generate_candidates (neighbors)** : this function is responsible of creating a list of candidtates and returning it based on a list of neighbors.
   
   **expand(state ,expand_list,i=0, diversification_limit=0.2)**: this function is responsible of recursively appending all the neigbors of the current state based on the possible actions to a list that is passed to it as a parameter.
  
   **expanded (state, diversification_limit=0.2)**: it applies the expand fucntion on the states that permits to have all the possible neighbors and returns them as a list.


In [19]:
import statistics
import stat
import copy
import json
import math
from scipy.stats import norm

def CalculateScore(statistical_variable, value):
    """
    Normalizes the score of a statistical variable (standard deviation, VaR, CVaR)
    to a value between 0 and 1 based on its risk level.
    """
    score_values = {"low": 0.3, "high": 0.7}
    level = LevelOfStatisticalVariable(statistical_variable, value)

    if "low" in level:
        min_val, max_val = level["low"]
        denominator = max_val - min_val if max_val != min_val else 1
        score = score_values["low"] * ((value - min_val) / denominator)

    elif "moderate" in level:
        min_val, max_val = level["moderate"]
        denominator = max_val - min_val if max_val != min_val else 1
        score = score_values["low"] + (score_values["high"] - score_values["low"]) * ((value - min_val) / denominator)

    elif "high" in level:
        min_val, max_val = level["high"]
        denominator = max_val - min_val if max_val != min_val else 1
        score = score_values["high"] + (1 - score_values["high"]) * ((value - min_val) / denominator)

    else:
        raise ValueError("Unrecognized level from LevelOfStatisticalVariable")

    return round(score, 4)


def CalculateTotalRisk(volatility_val,Var_val,Cvar_val):
    """
    calculates the total risk of the portfolio .
    """
    volatility_score = CalculateScore("standard_deviation",volatility_val)
    Var_score = CalculateScore("Var",Var_val)
    Cvar_score = CalculateScore("Cvar",Cvar_val)
    total_risk = (0.5 * volatility_score) + (0.4 * Var_score) + (0.1 * Cvar_score)
    return total_risk


def LevelOfStatisticalVariable(statistical_variable,value):
    """
      determines which level we can consider the statistical variable according to its value .
    """
    standard_deviation = [{"low" : (0,0.05)},
                          {"moderate" : (0.05,0.15)},
                          {"high" : (0.15,1)}]
    Var = [{"low" : (0,0.02)},
          {"moderate" : (0.02,0.05)},
          {"high" : (0.05,1)}]
    standard_deviation = [{"low" : (0,0.03)},
                          {"moderate" : (0.03,0.06)},
                          {"high" : (0.06,1)}]
    if statistical_variable == "standard_deviation":
      if value <= 0.05:
        return standard_deviation[0]
      if value > 0.05 and value < 0.15 :
        return standard_deviation[1]
      else:
        return standard_deviation[2]

    if statistical_variable == "Var":
      value = value * (-1)
      if value <= 0.02:
        return standard_deviation[0]
      if value > 0.02 and value < 0.05 :
        return standard_deviation[1]
      else:
        return standard_deviation[2]

    if statistical_variable == "Cvar":
      value = value * (-1)
      if value <= 0.03:
        return standard_deviation[0]
      if value > 0.03 and value < 0.06 :
        return standard_deviation[1]
      else:
        return standard_deviation[2]


def calculate_returns(prices):
    """Calculate daily returns from a list of prices."""
    returns = np.diff(prices) / prices[:-1]
    return returns


def get_var_and_cvar(stock, confidence_level=0.95):
    """
    Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR).

    Args:
        stock (dict): Dictionary containing 'symbol' and 'prices'.
        confidence_level (float): Confidence level for VaR (e.g., 0.95 for 95%).

    Returns:
        tuple: VaR and CVaR values.
    """
    prices =  stock["prices"]
    returns = calculate_returns(prices)

    z_score = norm.ppf(1 - confidence_level)  # Z-score for the given confidence level
    mean = np.mean(returns)
    std_dev = np.std(returns)
    var = mean - z_score * std_dev  # VaR formula

    # Calculate CVaR as the mean of losses exceeding VaR
    losses_beyond_var = returns[returns <= var]
    cvar = np.mean(losses_beyond_var) if len(losses_beyond_var) > 0 else var

    return var, cvar  # Return raw VaR/CVaR (negative for losses)



def GetStandarddeviation(node,user_constraints):  # we must modify the parameter to be self
    """
      calculates the standard deviation of the stock that represents the volatility of it .
      user constraint is a dictionary where {"preferred_risk":"-","investment_duration":"-"}
    """
    if user_constraints["investment_duration"] == "short_term":
      with open('Data\StockFilterData\stocks_consecutive_day_volatility.json','r') as file :
        data = json.load(file)
    else:
      with open('Data\StockFilterData\stocks_consecutive_day_annual_volatility.json','r') as file :
        data = json.load(file)
    symbol = node["stock"]
    stock_dict = next((item for item in data if item['symbol'] == symbol),None)  #next() finds the first match
    volatilities = stock_dict['volatilities']
    std_value = np.std(volatilities,ddof=1) #ddof is the data degree of freedom
    return std_value


def GetPortfolioStatVariables(node, portfolio): #update other functions to take a stock not a node
    """
	   calculates the statistical variables (standard deviation , var , cvar) for the portfolio .
    """
    quadratic_sum = 0
    var_sum = 0
    cvar_sum =0
    stat_variables = dict()
    stocks = portfolio.stocks
    for stock in stocks :
      std_dev = GetStandarddeviation(stock)
      var = get_var_and_cvar(stock)[0]
      cvar = get_var_and_cvar(stock)[1]
      weight = GetWeight(node,stock,portfolio)
      quadratic_sum += (weight ** 2) * (std_dev ** 2)
      var_sum += weight *  var
      cvar_sum += weight * cvar
    stat_variables["standard_deviation"] = math.sqrt(quadratic_sum)
    stat_variables["Var"] = var_sum
    stat_variables["Cvar"] = cvar_sum
    return stat_variables


def GetWeight(node,stock,portfolio):
    """
    computes the allocation amount of stock in the portfolio (budget based)
    """
    weight = (node["value"] *(stock["amt"] )) / portfolio.funds
    return weight


def allocation (node, portfolio):#-----------------------------
    """
    computes the allocation amount of stock in the portfolio (total return based)
    """
    value = node.stock["price"]* node.value
    port_return = portfolio.eval_por(portfolio.stocks)
    return value/port_return

def expan_bhd(stocks, portfolio , resdum, limit=0.2 ):
    """
    Expands the stock portfolio by evaluating whether to buy, sell, or hold each stock.

    :param stocks: List of stocks to evaluate.
    :param mainstock: The main stock object, which holds available funds.
    :param portfolio: The current portfolio that contains the user's owned stocks.
    :param limit: The threshold percentage limit for when a stock should be bought (default is 20%).
    :return: A list of actions to be taken (buy, sell, or hold) for each stock.
    """
    expand = []
    for stock in stocks:
        symbol = stock["symbol"]
        actions = {"hold": 0, "symbol": stock["symbol"]}  # Assuming stock has 'id' and 'price' attributes
        if portfolio.has_stock(symbol):  # Check if the portfolio already has the stock
            actions["sell"] = stock["price"] # Plan to sell the stock at its current price
        if portfolio.percentage(symbol) < limit and portfolio.funds >= stock["price"] :
            actions["buy"] = -stock["price"]  # Plan to buy the stock if the limit is exceeded
        expand.append(actions)
    return expand

def stock_class_limit_not_violated(portfolio , portfolio_type, mainstock,allocation):
    """
    checks if a class of stocks percentage is violating the pereferred stock type requirment (retrun False) or not (return True)
    """
    risky ,moderate ,lowrisk = classify_stock_risk(portfolio)
    risk_allocation_limit = {
     "aggressive" : {
         "high":   (0.50 , 0.70),
         "moderate":   (0.20 , 0.40),
         "low":   (0.00 , 0.15),
      },
      "moderate" : {
         "high":   (0.20 , 0.35),
         "moderate":   (0.4 , 0.50),
         "low":   (0.15 , 0.30),
      },
      "conservative" : {
         "high":   (0.00 , 0.10),
         "moderate":   (0.20 , 0.40),
         "low":   (0.50 , 0.70),
      }
      }
    if portfolio_type == "aggressive":
       if mainstock in risky :#imported from data
           return range_limit(risk_allocation_limit["aggressive"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["aggressive"]["moderate"],allocation)
       elif mainstock in lowrisk :#imported from data
           return range_limit(risk_allocation_limit["aggressive"]["low"],allocation)
    elif portfolio_type == "moderate":
       if mainstock in risky :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["moderate"],allocation)
       elif mainstock in lowrisk :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["low"],allocation)
    elif portfolio_type == "conservative":
       if mainstock in risky :#imported from data
           return range_limit(risk_allocation_limit["conservative"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["conservative"]["moderate"],allocation)
       elif mainstock in lowrisk :#imported from data
           return range_limit(risk_allocation_limit["conservative"]["low"],allocation)


def diversification_limit_not_violated( portfolio_type="aggressive", allocation=0.1):
   """
   checks if the diversification limit is violated (retrurns false) or not (returns true)
   """
   diversification_limit = {
         "aggressive":   (0.00 , 0.25),
         "moderate":   (0.00 , 0.15),
         "conservative":   (0.00 , 0.10),
         }
   if portfolio_type=="aggressive":
       return range_limit(diversification_limit["aggressive"] ,allocation)
   elif portfolio_type=="moderate":
       return range_limit(diversification_limit["moderate"] ,allocation)
   elif portfolio_type=="conservative":
       return range_limit(diversification_limit["conservative"] ,allocation)



def range_limit (tuple, value):
  """
  checks if a value is in the interval [tuple[0],tuple[1]]
  """
  x,y = tuple
  if x <= value <= y:
     return True
  else:
     return False



def TotalRiskLevel(risk_score):
  """
  determines the level of portfolio from the risk_score ("conservative" ,"moderate" or "aggressive" )
  """
  if risk_score<= 0.4 :
    return "conservative"
  elif risk_score <= 0.7:
    return "moderate"
  else:
    return "aggressive"

def number_of_invested(portfolio):
   """
   number of stocks that are being invested in in the portfolio
   """
   count = 0
   for stock in portfolio.stocks :
      if stock.amt >0 :
         count += 1
   return count

def calculate_cv(portfolio):
    """
    calculate the coeficient of variation of the number of shares
    """
    shares = [stock["amt"] for stock in portfolio.stocks]
    std = statistics.stdev  (shares) if len(shares) > 1 else 0
    meanshares =   statistics. mean(shares) if len(shares) > 1 else 0
    return std / meanshares if meanshares != 0 else int(0)

def expand(node, mainstock, portfolio , user_constraints , stocks):  # replace the node with self
    """
     user constraint is a dictionary where {"preferred_risk":"-","investment_duration":"-"}
    """

    expand = []
    default  = {"id": node["symbol"], "hold": 0}
    return expan_bhd(stocks, portfolio , user_constraints)

    portfolio.display_portfolio()
    expand.append(default)
    buy_portfolio = copy.deepcopy(portfolio) if len(portfolio.get_stocks() ) == 0 else None
    sell_portfolio= copy.deepcopy(portfolio) if len(portfolio.get_stocks() ) == 0 else None
    with open('/multiple_stocks_ratios.json', 'r') as file:
      data = json.load(file)

    for stockinfo in data :

       if stockinfo["symbol"] == mainstock["stock"]:
           print("Non",portfolio.get_stocks())
           for stock in portfolio.get_stocks():
              print("hi",)
              if stock["stock"] ==  mainstock["stock"]:
                  stock["amt"]+=1
                  print("hello")
                  buy_portfolio=copy.deepcopy(portfolio)
                  print("here",buy_portfolio)
                  stock["amt"]-=2
                  if  stock["amt"] >= 0 :
                      sell_portfolio = copy.deepcopy(portfolio)
                  stock["amt"]+=1 #back to its orignal value
    stat_variables = GetPortfolioStatVariables(node , buy_portfolio)
    std_value = stat_variables["standard_deviation"]
    var = stat_variables["Var"]
    cvar = stat_variables["Cvar"]

    #-----------------------------------------
    #Computing weight based on the variety of stocks in the portfolio
    #-----------------------------------------
    cv = calculate_cv(portfolio)
    numinv = number_of_invested(portfolio)
    if cv <= 0.5 and numinv >=6 :
       w= allocation(node, portfolio)
    else :
       w= GetWeight(node, mainstock,portfolio)
    #-----------------------------------------
    #Buy action check
    #-----------------------------------------
    if portfolio.funds >= node["value"]  :

      #diversification and stock class check
      portfolio_type = user_constraints["preferred_risk"]
      investement_duration = user_constraints["investment_duration"]
      portfolio_risk_score = CalculateTotalRisk(std_value,var,cvar)
      portfolio_risk = TotalRiskLevel(portfolio_risk_score)
      if  portfolio_risk == user_constraints["preferred_risk"]  and diversification_limit_not_violated(portfolio_type,w) and stock_class_limit_not_violated( portfolio , portfolio_type, mainstock,w) and investment_term_constraint( investement_duration,portfolio):
        expand.append({"id": node.symbol,"buy":-node.value})

    #-----------------------------------------
    #Sell action check
    #-----------------------------------------
    if sell_portfolio :
       portfolio_risk_score = CalculateTotalRisk(std_value,var,cvar)
       portfolio_risk = TotalRiskLevel(portfolio_risk_score)
       if portfolio_risk == user_constraints["preferred_risk"]  and diversification_limit_not_violated(portfolio_type,w) and stock_class_limit_not_violated( portfolio , portfolio_type, mainstock,w) and investment_term_constraint( investement_duration,portfolio):
           expand.append({"id": node.symbol,"sell":+node.value})
    return expand


def classify_stock_risk(portfolio):
    """
    classifies the portfolio stocks to three lists of risk levels
    """
    Risky=[]
    Moderate=[]
    LowRisk=[]

    with open ('Data\StockFilterData\multiple_stocks_ratios.json', 'r') as f :
       data = json.load(f)
    vol_list= extract_ratio_list (data,"volatility")
    cu_list= extract_ratio_list (data,"currentRatioTTM")
    dept_list= extract_ratio_list (data,"dept_to_equity")


    for stockitem in portfolio.stocks :
        stock_ratios = next((item for item in data if item["symbol"]==stockitem.stock),None)
        volatility = stock_ratios.get("volatility", 1.0)
        cu_ratio = stock_ratios.get("currentRatioTTM",15.0)
        debt_equity = stock_ratios.get("dept_to_equity",1.0)

        vol_nor = normalize(volatility,min(vol_list),max(vol_list))
        cu_nor = normalize(cu_ratio,min(cu_list),max(cu_list))
        dept_nor = normalize(debt_equity,min(dept_list),max(dept_list))

        class_val = vol_nor*0.7-cu_nor*0.3

        if class_val<0.3:
          LowRisk.append(stockitem)
        elif class_val<0.6:
          Moderate.append(stockitem)
        else:
          Risky.append(stockitem)
    return Risky,Moderate,LowRisk


def extract_ratio_list(data, key ,default=0.0 ):
    """
    extracts a list of a specefic ratio from data
    """
    return [item.get(key, default) for item in data if item.get(key) is not None   ]


def normalize (value, min , max):
    """
    normalize a value to the range [0,1]
    """
    if min == max:
       return 0.0
    return (value -min) / (max - min)

def investment_term_constraint( investement_duration,portfolio):
    #df = load_merged_data()
    #short_term = get_top_short_term(df, 30)
    #long_term = get_top_long_term(df, 30)
    shortcount=0
    longcount=0
    for stock in portfolio.stocks :
       if stock.stock in  short_term :
           shortcount+=1
       else :
           longcount+=1

    total_stocks= shortcount+longcount

    if total_stocks == 0:
        return True

    if investement_duration == "short" :
        if longcount/total_stocks > 0.2 :
          return False
    else :
        if shortcount/total_stocks > 0.2 :
          return False
    return True





- Greedy Search

In [20]:
import heapq
import inspect
import traceback

class MaxHeap:
    def __init__(self):
        self.heap = []
        self.counter = 0  # Used to break ties when priorities are equal

    def push(self, value, priority):
        """
        Push a value with its priority into the heap.
        Higher priority = served first.
        """
        try:
            # Store (negative priority, counter, value) to maintain stability and avoid comparing nodes
            heapq.heappush(self.heap, (-priority, self.counter, value))
            self.counter += 1
        except Exception as e:
            print(f"Error in push: {e}")

    def pop(self):
        """
        Remove and return the value with the HIGHEST priority.
        Returns (value, priority) or None if empty.
        """
        if not self.heap:
            return None
        neg_priority, _, value = heapq.heappop(self.heap)
        return (value, -neg_priority)  # Convert back to original priority

    def peek(self):
        """
        View (value, priority) of the highest-priority item without removal.
        Returns None if empty.
        """
        if not self.heap:
            return None
        neg_priority, _, value = self.heap[0]
        return (value, -neg_priority)

    @property
    def size(self):
        """Return the number of items in the heap."""
        return len(self.heap)

    def is_empty(self):
        """Check if the heap is empty."""
        return len(self.heap) == 0

In [21]:
import math
def normalize(value, k = 1) :
  return 1 - math.exp(-k * value)

def greedy_search(startNode, goal , portfolio ,prices):
    heap = MaxHeap()  # Assuming MaxHeap is correctly implemented
    heap.push(startNode, 0)
    visited = set()  # Using a set for O(1) lookups
    visited.add(f"{startNode.symbol}_{startNode.action}")  # Initial node

    while not heap.is_empty():
        current_node, val = heap.pop()

        if current_node.symbol == goal:  # Fixed attribute access
            return current_node

        val ,funds = evaluate(current_node ,portfolio , prices )
        key = current_node.get_Key()
        visited.add(key)
        for neighbor in current_node.neighbors:
            key_n = neighbor.get_Key()
            # Check if we can afford to buy AND we haven't visited this node
            if not (((funds < neighbor.value) and (neighbor.action == "buy")) or (key_n in visited)):
                heap.push(neighbor, neighbor.get_heu())


- A* search :

In [22]:
def A_star_search(startNode, goal,portfolio ,prices):
    heap = MaxHeap()
    heap.push(startNode, 0)
    visited = set()

    while not heap.is_empty():
        current_node, val = heap.pop()

        key_c = f"{current_node.symbol}_{current_node.action}"

        if current_node.symbol == goal:
            return current_node

        cost, funds = evaluate(current_node,portfolio,prices)

        for neighbor in current_node.neighbors:
            key_n = neighbor.get_Key()

            # Only push neighbor if it passes conditions and is not visited
            if not ((funds < neighbor.value and neighbor.action == "buy")):
                costn , fundsn = evaluate( neighbor,portfolio,prices)
                gn = 0.65 * neighbor.get_heu()+ 0.35 * normalize(costn*0.6+ funds*0.4)
                heap.push(neighbor, gn )

    return None


In [27]:

def search(stocks , start_date, end_date):
   T = get_stock_data(stocks, start_date, end_date)


   T_search = [[]for i in range(len(T[0]["prices"]))]
   for stock in T :
       for i in range(len(stock["prices"])):
         T_search[i].append({"symbol" : stock["symbol"] , "price" : stock["prices"][i] })

   return  T , T_search



---
3.Testing :
---

In [23]:
constraints = {"preferred_risk":"moderate","investment_duration":"short_term"}
def adapter(stocks , portfolio , constraints ):
  res = []
  for stock in stocks:
    main = {"stock": stock["symbol"], "price":  stock["price"], "amt":  portfolio.get_stock(stock["symbol"]) }
    node = {
        "symbol": stock["symbol"] ,
        "value": main["price"] * main["amt"] ,
        "stock": stock["symbol"] ,
        "eval_por": lambda stocks: sum(s["price"]* ( portfolio.get_stock(stock["symbol"]) ) for s in stocks)
    }
    ac_res  = expand(node, main, portfolio , constraints , stocks)

  return ac_res

In [24]:
def apply_acts(path, portfolio):
  for node in path :
    if node.action == "buy":
      portfolio.buy(node.symbol , node.value)
    elif node.action == "sell":
      portfolio.sell(node.symbol , node.value)


In [25]:
from copy import deepcopy
from datetime import datetime, timedelta
top_tech_stocks = [
    "AAPL",  # Apple - Tech giant with consistent innovation and revenue
    "MSFT",  # Microsoft - Dominates cloud and enterprise software
    "AMZN",  # Amazon - E-commerce and cloud computing leader
    "GOOGL", # Alphabet (Google) - Strong in AI, search, and YouTube
    "META",  # Meta Platforms - Social media and metaverse development
    "NVDA",  # NVIDIA - Leader in AI and graphics chips
    "TSLA",  # Tesla - EV pioneer with growing global presence
    "JPM",   # JPMorgan Chase - Top U.S. bank with solid returns
    "V",     # Visa - Dominant player in digital payments
    "PFE"    # Pfizer - Established pharmaceutical company with diverse pipeline
]

top_tech_short = [
    "GME",   # GameStop - Highly volatile, driven by retail traders
    "AMC",   # AMC Entertainment - Frequent meme stock with price swings
    "NVDA",  # NVIDIA - High demand, but often overbought/sold
    "PLTR",  # Palantir - Government contracts and data analytics, momentum-driven
    "RIVN",  # Rivian - EV startup with high speculation
    "SPY",   # S&P 500 ETF - Tracks the market, great for swing trading
    "QQQ",   # Nasdaq 100 ETF - Popular for day trading and momentum plays
    "XLF",   # Financial Select Sector SPDR Fund - Reacts quickly to economic news
    "TLRY",  # Tilray - Cannabis sector with frequent volatility
    "FUBO"   # FuboTV - High risk/reward, often influenced by news and trends
]

In [41]:
from copy import deepcopy
from datetime import datetime, timedelta

def get_time_frame(years=1, months=0):
    """Calculate date range based on years and months offset from pivot date (2023-01-01).

    Args:
        years: Number of years for the time frame (default 1)
        months: Additional months to add (default 0)

    Returns:
        tuple: (start_date, end_date) in 'YYYY-MM-DD' format

    Behavior:
        - With years=1: returns ('2023-01-01', '2024-01-01')
        - With years=2: returns ('2022-01-01', '2024-01-01')
        - Months are added to the end date
    """
    pivot_date = "2023-01-01"
    pivot = datetime.strptime(pivot_date, "%Y-%m-%d")

    # Calculate start date (years-1 before pivot)
    start = pivot - timedelta(days=365*(years-1))
    max_end = pivot + timedelta(days=365) + timedelta(days=30*months)
    end =  max_end

    return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

def get_past_frames(history, cutoff_index):
    """Return historical data up to the specified index"""
    return [
        {**stock, "prices": stock["prices"][0:cutoff_index]}
        for stock in history
    ]

def calculate_portfolio_value(portfolio, current_prices):
    """Calculate total value of the portfolio"""
    return sum(
        stock_data.amt * current_prices.get(stock_data.stock, 0)
        for stock_data in portfolio.stocks
    )

def run_simulation(stocks  = top_tech_stocks  , initial_capital=2000, years=1, months=0):
    """Main simulation function"""
    # Setup time frame and get historical data
    start_date, end_date = get_time_frame(years=years, months=months)

    short_term = False
    if years <  2 :
      short_term = True
    history, frames = search( stocks, start_date, end_date)
    # Initialize portfolio
    portfolio = Stocks(initial_capital)
    last_stock = stocks[-1]
    # Run time frame search
    for i in range(26, len(frames)):
        # Get historical data up to current point
        past_frames = get_past_frames(history, i)
        history_heuristic = {item["symbol"]: item for item in past_frames}
        expanded =adapter (frames[i],portfolio , constraints )
        nodes = setNodes(expanded, history_heuristic,  short_term )
        best_node = greedy_search(nodes[0], last_stock , portfolio, frames[i])
        apply_acts(get_path(best_node)[1:], portfolio)

    # Display results
    portfolio.display_portfolio()

    current_prices = {fr["symbol"]: fr["price"] for fr in frames[-1]}
    total_value = calculate_portfolio_value(portfolio, current_prices) + portfolio.get_funds()
    print("initial capite: " ,initial_capital )
    print("Total Portfolio Value:", total_value)
    print('return on investment : ' , ((total_value - initial_capital)/initial_capital) * 100 , "%")
    return total_value

# Example usage:
# 1 year simulation (default)
run_simulation(  top_tech_stocks   , 600 , 2)

# 2 year simulation with 3 additional months
# run_simulation(years=2, months=3)

<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[*********************100%***********************]  1 of 1 completed
<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[*********************100%***********************]  1 of 1 completed
<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[*********************100%***********************]  1 of 1 completed
<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[****

Bought 1 share of AAPL for $144.72. Remaining funds: $455.28
Bought 1 share of MSFT for $261.04. Remaining funds: $194.24
Bought 1 share of AMZN for $115.54. Remaining funds: $78.70
Bought 1 share of NVDA for $15.82. Remaining funds: $62.89
Bought 1 share of PFE for $45.52. Remaining funds: $17.36
Sold 1 share of MSFT for $250.37. Updated funds: $267.73
Sold 1 share of NVDA for $15.74. Updated funds: $283.47
Sold 1 share of PFE for $44.31. Updated funds: $327.78
Bought 1 share of MSFT for $253.92. Remaining funds: $73.86
Bought 1 share of NVDA for $17.30. Remaining funds: $56.57
Bought 1 share of PFE for $43.86. Remaining funds: $12.70
Sold 1 share of AAPL for $159.94. Updated funds: $172.65
Sold 1 share of MSFT for $273.79. Updated funds: $446.44
Sold 1 share of AMZN for $134.95. Updated funds: $581.39
Sold 1 share of NVDA for $18.14. Updated funds: $599.53
Sold 1 share of PFE for $43.25. Updated funds: $642.78
Bought 1 share of AAPL for $162.74. Remaining funds: $480.04
Bought 1 shar

874.917986869812

# A* search simulation :

In [37]:
from copy import deepcopy
from datetime import datetime, timedelta

def get_time_frame(years=1, months=0):
    """Calculate date range based on years and months offset from pivot date (2023-01-01).

    Args:
        years: Number of years for the time frame (default 1)
        months: Additional months to add (default 0)

    Returns:
        tuple: (start_date, end_date) in 'YYYY-MM-DD' format

    Behavior:
        - With years=1: returns ('2023-01-01', '2024-01-01')
        - With years=2: returns ('2022-01-01', '2024-01-01')
        - Months are added to the end date
    """
    pivot_date = "2023-01-01"
    pivot = datetime.strptime(pivot_date, "%Y-%m-%d")

    # Calculate start date (years-1 before pivot)
    start = pivot - timedelta(days=365*(years-1))
    max_end = pivot + timedelta(days=365) + timedelta(days=30*months)
    end =  max_end

    return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

def get_past_frames(history, cutoff_index):
    """Return historical data up to the specified index"""
    return [
        {**stock, "prices": stock["prices"][0:cutoff_index]}
        for stock in history
    ]

def calculate_portfolio_value(portfolio, current_prices):
    """Calculate total value of the portfolio"""
    return sum(
        stock_data.amt * current_prices.get(stock_data.stock, 0)
        for stock_data in portfolio.stocks
    )

def run_simulation(stocks = top_tech_stocks, initial_capital=2000, years=1, months=0):
    """Main simulation function"""
    # Setup time frame and get historical data
    start_date, end_date = get_time_frame(years=years, months=months)

    history, frames = search( stocks, start_date, end_date)


    short_term = False
    if years < 2 :
      short_term = True
    # Initialize portfolio
    portfolio = Stocks(initial_capital)
    last_stock = stocks[-1]
    # Run time frame search
    for i in range(26, len(frames)):
        # Get historical data up to current point
        past_frames = get_past_frames(history, i)
        history_heuristic = {item["symbol"]: item for item in past_frames}
        expanded =adapter (frames[i],portfolio , constraints )
        nodes = setNodes(expanded, history_heuristic,  short_term )
        best_node = A_star_search(nodes[0], last_stock , portfolio, frames[i])
        apply_acts(get_path(best_node)[1:], portfolio)

    # Display results
    portfolio.display_portfolio()

    current_prices = {fr["symbol"]: fr["price"] for fr in frames[-1]}
    total_value = calculate_portfolio_value(portfolio, current_prices) + portfolio.get_funds()
    print("initial capite: " ,initial_capital )
    print("Total Portfolio Value:", total_value)
    print('return on investment : ' , ((total_value - initial_capital)/initial_capital) * 100 , "%")
    return total_value

# Example usage:
# 1 year simulation (default)
run_simulation( top_tech_stocks , 500 , 1)

# 2 year simulation with 3 additional months
# run_simulation(years=2, months=3)

<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[*********************100%***********************]  1 of 1 completed
<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[*********************100%***********************]  1 of 1 completed
<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[*********************100%***********************]  1 of 1 completed
<ipython-input-13-2239476769>:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock, start=start_date, end=end_date, interval=interval)
[****

Bought 1 share of PFE for $38.06. Remaining funds: $461.94
Sold 1 share of PFE for $38.54. Updated funds: $500.47
Bought 1 share of GOOGL for $94.05. Remaining funds: $406.43
Bought 1 share of PFE for $38.63. Remaining funds: $367.80
Sold 1 share of PFE for $38.40. Updated funds: $406.19
Bought 1 share of PFE for $38.04. Remaining funds: $368.15
Sold 1 share of PFE for $37.72. Updated funds: $405.87
Bought 1 share of PFE for $37.95. Remaining funds: $367.92
Sold 1 share of PFE for $37.50. Updated funds: $405.42
Bought 1 share of AMZN for $95.79. Remaining funds: $309.63
Bought 1 share of PFE for $37.22. Remaining funds: $272.41
Bought 1 share of MSFT for $250.32. Remaining funds: $22.09
Sold 1 share of PFE for $37.15. Updated funds: $59.24
Bought 1 share of PFE for $36.67. Remaining funds: $22.57
Sold 1 share of PFE for $35.81. Updated funds: $58.38
Bought 1 share of PFE for $35.63. Remaining funds: $22.76
Sold 1 share of PFE for $35.29. Updated funds: $58.04
Bought 1 share of PFE for 

715.6524868011475